# FastFarmInterace: an MPI-based Python-FastFarm interface to develop wind farm control policies 

We start by defining the actuators we will control. There can be three of them: YAW, TORQUE or PITCH. For this purpose we define a dictionary where every key is the name of the control, with the allowed iteration:

```
controls = Dict(name_of_actuator : (mininum_value, maximum_value, maximum_step_at_each_iteration))
```

In [13]:
import numpy as np

from wfcrl.environments import data_cases as cases
from wfcrl.interface import FastFarmInterface

We chose the layout of the farm to simulate. The library comes with a list of pre-existing farm layouts that can be found in `environments/data_cases.py`. Let us use the `Farm6Fastfarm`, which has 6 turbines organized in 2 rows.

In [2]:
config = cases.Farm6Fastfarm()

In [3]:
config

6 turbines 
measurement_window: 200
	xcoords: [0.0, 504.0, 1008.0, 0.0, 504.0, 1008.0]
	ycoords: [-252, -252, -252, 252, 252, 252]
	dt: 3
log_file: log.txt

We will control the yaw of all turbines, allowing to vary between -20 and 20°, with a maximum change between two iterations of 15°. We control the pitch between 0° and 45°, with a maximum step of 1°.

In [11]:
print(config.interface_kwargs)

{'measurement_window': 200, 'simul_kwargs': {'xcoords': [0.0, 504.0, 1008.0, 0.0, 504.0, 1008.0], 'ycoords': [-252, -252, -252, 252, 252, 252], 'dt': 3}, 'log_file': 'log.txt'}


In [9]:
num_iter = 100
ff_interface = FastFarmInterface(
    num_turbines=config.n_turbines,
    max_iter=num_iter,
    **config.interface_kwargs
)

Deltas: [2.10730392e-05 9.06140684e-04 1.81228137e-03 2.10730392e-05
 9.06140684e-04 1.81228137e-03]
Exception has been raise. I put this print statement instead. Check with EB.
Exception: Some X0_High are not on an integer multiple of the high-res grid
Created FAST.Farm input file: simulators/fastfarm/inputs/FarmInputs/Case.fstf


To change the yaw, pitch or torque command, use the method `update_command(yaw, pitch, torque)`. Let's control the yaw and pitch:

In [17]:
ff_interface.reset()
for i in range(num_iter):
    print("PYTHON: Iter ", i)
    yaw_command = np.zeros(ff_interface.num_turbines, dtype=np.double)
    pitch_command = np.zeros(ff_interface.num_turbines, dtype=np.double)
    if i > 20:
        yaw_command[0] = 30
        pitch_command[0] = 1
    ff_interface.update_command(yaw=yaw_command, pitch=pitch_command)
    print(f"Iter {i} - Sent command YAW {ff_interface.get_yaw_command()} - "
          f" PITCH {ff_interface.get_pitch_command()}"
          f" TORQUE {ff_interface.get_torque_command()}\n"
          f" Received Power: {ff_interface.get_turbine_powers()}"
          f" Wind : {ff_interface.get_turbine_wind()}\n"
    )

Spawning process simulators/fastfarm/bin/FAST.Farm_x64_OMP_2023.exe simulators/fastfarm/inputs/FarmInputs/Case.fstf
Interface: will receive 12 measures at every iteration
PYTHON: Iter  0
Iter 0 - Sent command YAW [0. 0. 0. 0. 0. 0.] -  PITCH [0. 0. 0. 0. 0. 0.] TORQUE None -  Received Power: [1681373.75  1745663.    1569152.875 1544478.125 1597603.25  1651373.75 ] Wind : [  8.74695969 276.0602417 ]

PYTHON: Iter  1
Iter 1 - Sent command YAW [0. 0. 0. 0. 0. 0.] -  PITCH [0. 0. 0. 0. 0. 0.] TORQUE None -  Received Power: [1725271.1875 1822723.6875 1588757.375  1551444.5    1625316.875
 1681648.0625] Wind : [  8.74695969 276.0602417 ]

PYTHON: Iter  2
Iter 2 - Sent command YAW [0. 0. 0. 0. 0. 0.] -  PITCH [0. 0. 0. 0. 0. 0.] TORQUE None -  Received Power: [1759347.125      1881772.91666667 1610341.75       1540417.
 1641216.         1699228.875     ] Wind : [  8.70515474 273.75221761]

PYTHON: Iter  3
Iter 3 - Sent command YAW [0. 0. 0. 0. 0. 0.] -  PITCH [0. 0. 0. 0. 0. 0.] TORQUE None -